# Credit Risk Resampling Techniques

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [ ]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Read the CSV and Perform Basic Data Cleaning

In [ ]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [ ]:
# Load the data
file_path = '/Users/user/Downloads/Module-17-Challenge-Resources/LoanStats_2019Q1.csv'
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

# get_dummies

In [ ]:
# Create our features
X = df[columns]
X = X.drop(target, axis=1)

X = pd.get_dummies(X, columns=["home_ownership","verification_status","issue_d", "pymnt_plan","initial_list_status",
                                "next_pymnt_d","application_type","hardship_flag", "debt_settlement_flag"])
# Create our target
y = df[target]

In [ ]:
X.describe()

In [ ]:
# Check the balance of our target values
y['loan_status'].value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
# YOUR CODE HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# BalancedRandomForestClassifier

In [ ]:
brf = BalancedRandomForestClassifier(n_estimators=100,random_state=1)
brf.fit(X_train_scaled, y_train)


In [ ]:
predictions = brf.predict(X_test_scaled)
balanced_accuracy_score(y_test, predictions)

In [ ]:
confusion_matrix(y_test, predictions)

In [ ]:
print(classification_report_imbalanced(y_test, predictions))

In [ ]:
feature_importances = brf.feature_importances_
features = sorted(zip(X.columns, brf.feature_importances_), key = lambda x: x[1],reverse=True)
for feature in features:
    print(f'{feature[0]}: ({feature[1]})')

# EasyEnsembleClassifier

In [ ]:
eec = EasyEnsembleClassifier(random_state=1,n_estimators=100)
eec.fit(X_train_scaled, y_train) 

In [ ]:
predictions = eec.predict(X_test_scaled)
balanced_accuracy_score(y_test, predictions)

In [ ]:
confusion_matrix(y_test, predictions)

In [ ]:
print(classification_report_imbalanced(y_test, predictions))